Предсказание результатов игроков.
Файл для отправки на проверку пишется в папку data/submission/.

In [8]:
import pandas as pd
from sklearn.base import clone
from sklearn.metrics import recall_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.model_selection import KFold

In [9]:
assess_cols = ['Analytical thinking', 'Systemic thinking', 'Adaptability', 'Focus']

In [10]:
# Загружаем данные
users_df = pd.read_csv('../data/raw/user.csv', index_col='user_id')
results_df = pd.read_csv('../data/raw/results.csv', index_col='id')
submit_df = pd.read_csv('../data/raw/to_submit.csv', index_col='id')

teammate_assessments_df = pd.read_csv('../data/preprocessed/teammate_assessments.csv', index_col='user_id')
team_results_df = pd.read_csv('../data/preprocessed/user_with_team_results.csv', index_col='user_id')
decisions_df = pd.read_csv('../data/preprocessed/user_decision_one_hot_all.csv', index_col='user_id')
# decisions_df = pd.read_csv('../data/preprocessed/user_decision_period_one_hot_all.csv', index_col='user_id')

In [11]:
# Баллы, полученные коллегами по команде
users_df = users_df.merge(teammate_assessments_df, left_index=True, right_index=True)
# Результаты команды игрока
users_df = users_df.merge(team_results_df, left_index=True, right_index=True)
# Принимал или нет игрок решение
users_df = users_df.merge(decisions_df, left_index=True, right_index=True)

In [12]:
users_df.head()

,team_id,game_id,Analytical thinking mates,Systemic thinking mates,Adaptability mates,Focus mates,score_1_0,place_1_0,score_1_1,place_1_1,...,347,9,8,216,231,7,233,346,261,258
user_id,,,,,,,,,,,,,,,,,,,,,
10189,1664,235,4.0,4.0,4.0,4.0,4,5,4,2,...,False,False,False,False,False,False,False,False,False,False
10018,1690,237,5.0,5.0,5.0,5.0,4,2,6,2,...,False,False,False,False,False,False,False,False,False,False
10580,1394,194,4.0,4.0,4.0,4.0,4,4,6,3,...,False,False,False,False,False,False,False,False,False,False
10600,1908,262,5.0,5.0,5.0,5.0,4,1,14,1,...,False,False,False,False,False,False,False,False,False,False
10112,1592,219,4.0,4.0,4.0,4.0,4,2,3,3,...,False,False,False,False,False,False,False,False,False,False


In [13]:
# Игроки с известными оценками
users_known_df = results_df[[]].merge(users_df, left_index=True, right_index=True)
# Игроки для которых надо предсказать результать 
users_test_df = submit_df[[]].merge(users_df, left_index=True, right_index=True)

In [14]:
kf = KFold(n_splits=4, shuffle=True, random_state=42)
clf = RandomForestClassifier(
    random_state=42,
    n_estimators=5000,
    # criterion='entropy',
    # max_depth=10,
    # max_features=0.5,
    # class_weight='balanced',
    )
recall = 0
to_submit_df = submit_df[[]].copy()

for column in assess_cols:
    # Оценка точности предсказание через K-fold.
    for train_index, valid_index in kf.split(users_known_df):
        X_train, X_valid = users_known_df.iloc[train_index,:], users_known_df.iloc[valid_index,:]
        y_train, y_valid = results_df.iloc[train_index,:][column], results_df.iloc[valid_index,:][column]
        model = clone(clf).fit(X_train, y_train)
        pred = model.predict(X_valid)
        recall += recall_score(y_valid, pred, labels=[1.0,2.0,3.0,4.0,5.0,6.0], average='micro')
    # Построение модели для финального предсказания на основе всех доступных данных
    model_full = clone(clf).fit(users_known_df, results_df[column])
    to_submit_df[column] = model_full.predict(users_test_df)
    # print(f'Model for column {column}: {model.__dict__}')
# Делим суммарную точность на число оценок (4), число полей K-fold, предполагаем что паблик выборка составляет половину от всей тестовой
print(recall/4/kf.n_splits/2)
to_submit_df.to_csv('../data/submission/random_forest.csv')

0.3056985294117647
